<a href="https://colab.research.google.com/github/vermaAstra/python-docs-samples/blob/main/geminidataanalytics/colabs/geminidataanalytics_public_SDK_demo_%7C_vPublished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copyright 2025 Google LLC.
# SPDX-License-Identifier: Apache-2.0

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Background and Overview
The **Conversational Analytics API** lets you chat with your BigQuery or Looker data anywhere, including embedded Looker dashboards, Slack and other chat apps, or even your own web applications. Your team members can get answers where they need them, when they need them, in the applications they use every day. You can find the [Colab example with the Python SDK here](https://colab.sandbox.google.com/drive/1UF1I4ZGU662R74vbNwk-wtmVWpQl107h?resourcekey=0-vQ8nXlhIpj87G65FbkjG8A#scrollTo=gBy8t2OiR73i).

Please provide feedback to conversational-analytics-api-feedback@google.com

This notebook will help you
1. Setting up the SDK
2. Authenticate to GCP
3. Add data
3. Perform agent operations (create, list, get, delete)
4. Manage conversations (create, list, get)
5. Ask questions with your agent

## Disclaimers
- Please **make a copy** of this notebook. Do not edit it directly
- This is a **Pre-GA** product. See [documentation](https://cloud.google.com/gemini/docs/conversational-analytics-api/overview) for more details.
- This product is currently under allowlist and not publicly available. Please contact your account team if you have questions.


## API Enablement

**Please fill in the billing_project form field with your own GCP project.  The project must have the following APIs enabled:**
-  [cloudaicompanion API](https://console.cloud.google.com/apis/library/cloudaicompanion.googleapis.com)
-  [Gemini Data Analytics API](https://console.cloud.google.com/apis/library/geminidataanalytics.googleapis.com)
-  [BQ API](https://console.cloud.google.com/marketplace/product/google/bigquery.googleapis.com)
-  [Dataform API](https://console.cloud.google.com/apis/library/dataform.googleapis.com)
- [Vertex AI API](https://console.cloud.google.com/apis/library/aiplatform.googleapis.com)

You may pass in any BigQuery project/dataset/table for which you have read permissions.




# Setup (Choose Option 1 or 2, run once, and restart the Colab runtime when prompted)
- Option 1: Download and install the pre-built SDK from Google Drive
- Option 2: [Currently not available] Build the client library from the GitHub repo and install

## Option 1: Download and install the pre-built SDK from Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# You should have been shared this folder

%cp -R /content/drive/Shareddrives/geminidataanalytics/SDKs/geminidataanalytics-v1alpha-py.tar.gz /tmp/geminidataanalytics-v1alpha-py.tar.gz

%cd /tmp
!tar -xvzf geminidataanalytics-v1alpha-py.tar.gz
%cd geminidataanalytics-v1alpha-py

# You'l be asked to restart the runtime after this finishes.  After restarting there's no need to run this again.
!pip install .

# Setup

## Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Imports

In [ ]:
from pygments import highlight, lexers, formatters
import pandas as pd
import requests
import json as json_lib
import altair as alt
import IPython
from IPython.display import display, HTML

import proto
from google.protobuf.json_format import MessageToDict, MessageToJson


## Define utilities for viewing responses

In [ ]:
def handle_text_response(resp):
  parts = getattr(resp, 'parts')
  print(''.join(parts))

def display_schema(data):
  fields = getattr(data, 'fields')
  df = pd.DataFrame({
    "Column": map(lambda field: getattr(field, 'name'), fields),
    "Type": map(lambda field: getattr(field, 'type'), fields),
    "Description": map(lambda field: getattr(field, 'description', '-'), fields),
    "Mode": map(lambda field: getattr(field, 'mode'), fields)
  })
  display(df)

def display_section_title(text):
  display(HTML('<h2>{}</h2>'.format(text)))

def format_looker_table_ref(table_ref):
 return 'lookmlModel: {}, explore: {}, lookerInstanceUri: {}'.format(table_ref.lookml_model, table_ref.explore, table_ref.looker_instance_uri)

def format_bq_table_ref(table_ref):
  return '{}.{}.{}'.format(table_ref.project_id, table_ref.dataset_id, table_ref.table_id)

def display_datasource(datasource):
  source_name = ''
  if 'studio_datasource_id' in datasource:
   source_name = getattr(datasource, 'studio_datasource_id')
  elif 'looker_explore_reference' in datasource:
   source_name = format_looker_table_ref(getattr(datasource, 'looker_explore_reference'))
  else:
    source_name = format_bq_table_ref(getattr(datasource, 'bigquery_table_reference'))

  print(source_name)
  display_schema(datasource.schema)

def handle_schema_response(resp):
  if 'query' in resp:
    print(resp.query.question)
  elif 'result' in resp:
    display_section_title('Schema resolved')
    print('Data sources:')
    for datasource in resp.result.datasources:
      display_datasource(datasource)

def handle_data_response(resp):
  if 'query' in resp:
    query = resp.query
    display_section_title('Retrieval query')
    print('Query name: {}'.format(query.name))
    print('Question: {}'.format(query.question))
    print('Data sources:')
    for datasource in query.datasources:
      display_datasource(datasource)
  elif 'generated_sql' in resp:
    display_section_title('SQL generated')
    print(resp.generated_sql)
  elif 'result' in resp:
    display_section_title('Data retrieved')

    fields = [field.name for field in resp.result.schema.fields]
    d = {}
    for el in resp.result.data:
      for field in fields:
        if field in d:
          d[field].append(el[field])
        else:
          d[field] = [el[field]]

    display(pd.DataFrame(d))

def handle_chart_response(resp):
  def _value_to_dict(v):
    if isinstance(v, proto.marshal.collections.maps.MapComposite):
      return _map_to_dict(v)
    elif isinstance(v, proto.marshal.collections.RepeatedComposite):
      return [_value_to_dict(el) for el in v]
    elif isinstance(v, (int, float, str, bool)):
      return v
    else:
      return MessageToDict(v)

  def _map_to_dict(d):
    out = {}
    for k in d:
      if isinstance(d[k], proto.marshal.collections.maps.MapComposite):
        out[k] = _map_to_dict(d[k])
      else:
        out[k] = _value_to_dict(d[k])
    return out

  if 'query' in resp:
    print(resp.query.instructions)
  elif 'result' in resp:
    vegaConfig = resp.result.vega_config
    vegaConfig_dict = _map_to_dict(vegaConfig)
    alt.Chart.from_json(json_lib.dumps(vegaConfig_dict)).display();

def show_message(msg):
  m = msg.system_message
  if 'text' in m:
    handle_text_response(getattr(m, 'text'))
  elif 'schema' in m:
    handle_schema_response(getattr(m, 'schema'))
  elif 'data' in m:
    handle_data_response(getattr(m, 'data'))
  elif 'chart' in m:
    handle_chart_response(getattr(m, 'chart'))
  print('\n')


# Example API call
**All API calls** take `billing_project` (a GCP billing project), a `question` (first in a list of Messages), and `system_instruction` (optional)


**BigQuery calls** create a BigQueryTableReference contain: via `project_id`, `dataset_id`, and `table_id`. Note that multiple table references can be passed in a list.


**Looker calls** create a LookerExploreReference with `looker_instance_uri`, `lookml_model`, and `explore`. Auth credentials are created with `client_id` and `client_secret`.

*Note: options for advanced system instructions are in the cells below*

In [ ]:
# @title Import the GeminiDataAnalytics client library

from google.cloud import geminidataanalytics
from google.iam.v1 import iam_policy_pb2, policy_pb2

In [ ]:
# @title Data Source details And Client

data_agent_client = geminidataanalytics.DataAgentServiceClient()
data_chat_client = geminidataanalytics.DataChatServiceClient()

################### Billing Project ###################
billing_project = "cloud-raspberry-e2e-testing" # @param {type:"string"}


# ################### BigQuery Data Source(s) ###################
# # List datasource(s) you'd like to ask questions of
# # This example only includes one source but you can include a list.
# # Make sure you have permission to query any tables listed here.
bq_project_id = "bigquery-public-data" # @param {type:"string"}
bq_dataset_id = "faa" # @param {type:"string"}
bq_table_id = "us_airports" # @param {type:"string"}


################## Looker Data Source ###################
# Alternatively you can specify Looker datasources (uncomment to use)
# An example with: https://my_company.looker.com/explore/my_model/my_explore
looker_client_id = "<add client_id here>" # @param {type:"string"}
looker_client_secret = "<add client_secret here>" # @param {type:"string"}
looker_access_token = "<add access_token here>" # @param {type:"string"}
looker_instance_uri = "https://my_company.looker.com" # @param {type:"string"}
lookml_model = "<add lookml_model here>" # @param {type:"string"}
explore = "<add explore here>" # @param {type:"string"}


################## Looker Studio Data Source ###################
studio_datasource_id = "<add studio_datasource_id here>" # @param {type:"string"}


### Provide Robust System Instruction
- Providing more context around your business and your data improves the quality of answers. We recommend (though do not require) you follow our example `system_instruction` below and fill out the stringified YAML **template** provided with field instructions, validated ("golden") queries, relationships, business terms, etc.
- We are working to improve how our product ingests and uses this context to improve question-answering accuracy

In [ ]:
###############################################################################
#######         Define system instruction for the question.             #######
#######    system_instruction is optional, can be used to steer         #######
#######              the agent's answering strategy.                    #######
###############################################################################

###############################################################################
##### Example template for system instruction for a sales analyst agent:  #####
###############################################################################

# """
# - system_instruction: >-
#     You are an expert sales analyst and understand how to answer questions about
#     the sales data for a fictitious e-commence store
# - tables:
#     - table:
#         - name: bigquery-public-data.thelook_ecommerce.orders
#         - description: orders for The Look fictitious e-commerce store.
#         - synonyms: sales
#         - tags: 'sale, order, sales_order'
#         - fields:
#             - field:
#                 - name: order_id
#                 - description: unique identifier for each order
#             - field:
#                 - name: user_id
#                 - description: unique identifier for each user
#             - field:
#                 - name: status
#                 - description: status of the order
#                 - sample_values:
#                     - complete
#                     - shipped
#                     - returned
#             - field:
#                 - name: gender
#                 - description: gender of the user
#                 - sample_values:
#                     - male
#                     - female
#             - field:
#                 - name: created_at
#                 - description: date and time when the order was created in timestamp format
#             - field:
#                 - name: returned_at
#                 - description: >-
#                     date and time when the order was returned in timestamp
#                     format
#             - field:
#                 - name: num_of_item
#                 - description: number of items in the order
#                 - aggregations: 'sum, avg'
#             - field:
#                 - name: earnings
#                 - description: total sales from the order
#                 - aggregations: 'sum, avg'
#             - field:
#                 - name: cost
#                 - description: total cost to get the items for the order
#                 - aggregations: 'sum, avg'
#         - measures:
#             - measure:
#                 - name: profit
#                 - description: raw profit
#                 - exp: cost - earnings
#                 - synonyms: gains
#         - golden_queries:
#             - golden_query:
#                 - natural_language_query: How many orders are there?
#                 - sql_query: SELECT COUNT(*) FROM sqlgen-testing.thelook_ecommerce.orders
#             - golden_query:
#                 - natural_language_query: How many orders were shipped?
#                 - sql_query: >-
#                     SELECT COUNT(*) FROM sqlgen-testing.thelook_ecommerce.orders
#                     WHERE status = 'shipped'
#         - golden_action_plans:
#             - golden_action_plan:
#                 - natural_language_query: Show me the number of orders broken down by gender.
#                 - action_plan:
#                     - step: >-
#                         Run a SQL query on the table
#                         sqlgen-testing.thelook_ecommerce.orders to get a
#                         breakdown of order count by gender.
#                     - step: >-
#                         Create a vertical bar plot using the retrieved data,
#                         with one bar per gender.
#     - table:
#         - name: sqlgen-testing.thelook_ecommerce.users
#         - description: user of The Look fictitious e-commerce store.
#         - synonyms: customers
#         - tags: 'user, customer, buyer'
#         - fields:
#             - field:
#                 - name: id
#                 - description: unique identifier for each user
#             - field:
#                 - name: first_name
#                 - description: first name of the user
#                 - tag: person
#                 - sample_values: 'graham, sara, brian'
#             - field:
#                 - name: last_name
#                 - description: first name of the user
#                 - tag: person
#                 - sample_values: 'warmer, stilles, smith'
#             - field:
#                 - name: gender
#                 - description: gender of the user
#                 - sample_values:
#                     - male
#                     - female
#             - field:
#                 - name: email
#                 - description: email of the user
#                 - tag: contact
#                 - sample_values: 'brian@gmail.com, graham@gmail.com'
#         - golden_queries:
#             - golden_query:
#                 - natural_language_query: How many unique customers are there?
#                 - sql_query: >-
#                     SELECT COUNT(DISTINCT id) FROM
#                     sqlgen-testing.thelook_ecommerce.users
#             - golden_query:
#                 - natural_language_query: How many female users have gmail email id?
#                 - sql_query: >-
#                     SELECT COUNT(DISTINCT id) FROM
#                     sqlgen-testing.thelook_ecommerce.users WHERE users.gender =
#                     'F' AND users.email LIKE '%@gmail.com';
#     - relationships:
#         - relationship:
#             - name: earnings_to_user
#             - description: >-
#                 Sales table is related to the users table and can be joined for
#                 aggregated view.
#             - relationship_type: many-to-one
#             - join_type: left
#             - left_table: str sqlgen-testing.thelook_ecommerce.orders
#             - right_table: sqlgen-testing.thelook_ecommerce.users
#             - relationship_columns: '// Join columns - left_column:''user_id'' - right_column:''id'''
# - glossaries:
#     - glossary:
#         - term: male
#         - description: male gender
#     - glossary:
#         - term: female
#         - description: female gender
#     - glossary:
#         - term: complete
#         - description: complete status
#         - synonyms: 'finish, done, fulfilled'
#     - glossary:
#         - term: shipped
#         - description: shipped status
#     - glossary:
#         - term: returned
#         - description: returned status
#     - glossary:
#         - term: OMPF
#         - description: Order Management and Product Fulfillment
# - additionl_descriptions:
#     - text: All the sales data is for Looker organization.
#     - text: 'Orders can be of three categories, food , clothes, electronics.'
# """

###############################################################################
######    Use the schema below to provide your own system description.   ######
###############################################################################

# """
# - system_description: str # Expected behavior of the agent. Eg. You are a sales analyst.
# - tables:
#     - table: # Table relevant for this agent.
#         - name: str # Name of the table.
#         - description: str # Description of the table.
#         - synonyms: list[str] # List of synonyms used to refer to the table.
#         - tags: list[str] # List of tags associated with the table.
#         - fields: # Fields in the table.
#             - field:
#             - name: str # Name of the column.
#             - description: str # Description of the column.
#             - synonyms: list[str] # List of synonyms used to refer to the column.
#             - tags: list[str] # List of tags associated with the column.
#             - sample_values: list[str] # List of sample values in the column.
#             - aggregations: list[str] # Any commonly used or default aggregations associated with the column.
#         - measures: # Measures for the table.
#             - measure:
#                 - name: str # Name of the measure.
#                 - description: str # Description of the meansure.
#                 - exp: str # Expression to construct the measure.
#                 - synonyms: list[str] # List of synonyms used to refer to the measure.
#         - golden_queries: # Golden or popular queries for the table.
#             - golden_query:
#                 - natural_language_query: str # Natural language query.
#                 - sql_query: str # SQL query.
#         - golden_action_plans: # Golden action plans as the suggested steps to take (in order) to answer the query.
#           - golden_action_plan:
#             - natural_language_query: str # Natural language query.
#             - action_plan:
#               - step: str # Step to take.
#     - relationships: # Join relationships between tables.
#         - relationship:
#           - name: str # Name of the relationship.
#           - description: str # Description of the relationship.
#           - relationship_type: str # Relationship type: one-to-one, one-to-many, many-to-one, many-to-many.
#           - join_type: str # Join type: inner, outer, left, right, full
#           - left_table: str # Left table name.
#           - right_table: str # Right table name.
#           - relationship_columns: # Join columns.
#               - left_column: str # Join column from left table.
#               - right_column: str # Join column from right table.
# - glossaries: # Business glossary, jargon, etc.
#     - glossary:
#         - term: str # Name of the term. Term can be a word, phrase, abbreviation, etc.
#         - description: str # Description or definition of the term.
#         - synonyms: list[str] # List of synonyms for the term.
# - additional_descriptions:
#     - text: str # Any additional description that was not covered above.
# """

In [ ]:
##################### OPTION 1 ###############################
#######                                                #######
#######        Fill the above YAML template for        #######
#######  system_instruction as shown in example above. #######
#######                                                #######
##############################################################

system_instruction = """"""

In [ ]:
##################### OPTION 2 ###############################
#######                                                #######
#######         Leave system_instruction empty         #######
#######         or provide description in form         #######
#######                 of long string.                #######
#######                                                #######
##############################################################

# Define context for question.
# system_instruction is optional, can be used to steer the agent's answering
# strategy.
system_instruction = "Think like an Analyst" # @param {type:"string"}

### *Helper functions for processing the system_instruction yaml string*

In [ ]:
import yaml
from yaml.scanner import ScannerError
from collections import Counter
import re

def cleanup_path(input_string):
  """Removes bracketed numbers and preceding dots from an input string.

  Args:
    input_string: The input string.

  Returns:
    The string with bracketed numbers and preceding dots removed.
  """
  cleaned_string = re.sub(r"\[\d+]", "", input_string)
  if cleaned_string.startswith("."):
    cleaned_string = cleaned_string[1:]
  return cleaned_string

def is_valid_yaml(yaml_string):
    try:
        yaml.safe_load(yaml_string)
        return True
    except ScannerError:
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

def print_counter_desc(data):
    # Count the occurrences of each item in the list.
    item_counts = Counter(data)

    # Sort the items in descending order based on their counts.
    sorted_items = sorted(item_counts.items(), key=lambda item: item[1], reverse=True)

    # Print the items and their counts.
    for item, count in sorted_items:
        print(f"{item}: {count}")

def extract_insights_from_system_instruction(yaml_string):

  """
    Extracts insights into used fields from a YAML string.

    Args:
      yaml_string: The YAML string to analyze.
  """

  if not is_valid_yaml(yaml_string):
      print("Received Invalid or not a YAML format. Cannot extract insights.")
      return

  try:
      data = yaml.safe_load(yaml_string)
      used_fields = []
      def traverse(node, path=""):
            if isinstance(node, dict):
                for key, value in node.items():
                    new_path = f"{path}.{key}" if path else key
                    traverse(value, new_path)
            elif isinstance(node, list):
                for i, item in enumerate(node):
                    new_path = f"{path}[{i}]"
                    traverse(item, new_path)
            else:
                used_fields.append(cleanup_path(path))

      traverse(data)
      print("Insights:")
      print("Number of unique fields used: ", len(set(used_fields)))
      print("Used fields in YAML:")
      print()
      print_counter_desc(used_fields)

  except ScannerError as e:
      print(f"Invalid YAML: {e}")
      return None
  except Exception as e:
      print(f"An unexpected error occurred: {e}")
      return None


# Printing insights assuming 'system_instruction' is defined above.
extract_insights_from_system_instruction(system_instruction)

Insights:
Number of unique fields used:  1
Used fields in YAML:

: 1


### Datasource set-up

In [ ]:
# BigQuery Data Source
bigquery_table_reference = geminidataanalytics.BigQueryTableReference()
bigquery_table_reference.project_id = bq_project_id
bigquery_table_reference.dataset_id = bq_dataset_id
bigquery_table_reference.table_id = bq_table_id

# Looker Data source
looker_explore_reference = geminidataanalytics.LookerExploreReference()
looker_explore_reference.looker_instance_uri = looker_instance_uri
looker_explore_reference.lookml_model = lookml_model
looker_explore_reference.explore = explore

# When connecting to the Lokker datasource, you can authenticate using either:
# 1. client_id and client_secret
# 2. access_token
credentials = geminidataanalytics.Credentials()
credentials.oauth.secret.client_id = looker_client_id
credentials.oauth.secret.client_secret = looker_client_secret
# Uncomment the next line and comment out the above two lines if you are using access_token for authentication
# credentials.oauth.token.access_token = looker_access_token


# Looker Studio
studio_references = geminidataanalytics.StudioDatasourceReference()
studio_references.datasource_id = studio_datasource_id


# Connect to your Data Source
datasource_references = geminidataanalytics.DatasourceReferences()
datasource_references.bq.table_references = [bigquery_table_reference]
# datasource_references.looker.explore_references = [looker_explore_reference]
# datasource_references.studio.studio_references = [studio_references]


# Context set-up for stateful chat
published_context = geminidataanalytics.Context()
published_context.system_instruction = system_instruction
published_context.datasource_references = datasource_references
published_context.options.analysis.python.enabled = True # if wanting to use advanced analysis with python


# Context set-up for stateless chat
# datasource_references.looker.credentials = credentials # uncomment this if you are using looker datasource
inline_context = geminidataanalytics.Context()
inline_context.system_instruction = system_instruction
inline_context.datasource_references = datasource_references
inline_context.options.analysis.python.enabled = True # if wanting to use advanced analysis with python

### Create and Get Data Agent

In [ ]:
# @title Create Data Agent

data_agent_id = "data_agent_bq_0807" # @param {type:"string"}

data_agent = geminidataanalytics.DataAgent()
data_agent.data_analytics_agent.published_context = published_context
data_agent.name = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}" #Optional

request = geminidataanalytics.CreateDataAgentRequest(
    parent=f"projects/{billing_project}/locations/global",
    data_agent_id=data_agent_id, #Optional
    data_agent=data_agent,
)

try:
    response = data_agent_client.create_data_agent(request=request)
    print(f"Data Agent created:\n\n{response.metadata}")
except Exception as e:
    print(f"Error creating Data Agent: {e}")


Error creating Data Agent: 409 Resource 'projects/cloud-raspberry-e2e-testing/locations/global/dataAgents/data_agent_bq_0807' already exists [resource_name: "projects/cloud-raspberry-e2e-testing/locations/global/dataAgents/data_agent_bq_0807"
]


In [ ]:
# @title Get Data Agent

# Initialize request argument(s)
data_agent_id = "data_agent_bq_0807" # @param {type:"string"}

request = geminidataanalytics.GetDataAgentRequest(
    name=f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}",
)

# Make the request
response = data_agent_client.get_data_agent(request=request)

# Handle the response
print(response)

name: "projects/cloud-raspberry-e2e-testing/locations/global/dataAgents/data_agent_bq_0807"
description: "This is the description of data_agent."
create_time {
  seconds: 1752041886
  nanos: 260681915
}
update_time {
  seconds: 1752041886
  nanos: 973576585
}
data_analytics_agent {
  published_context {
    system_instruction: "Think like an Analyst"
    options {
      analysis {
        python {
          enabled: true
        }
      }
    }
    datasource_references {
      bq {
        table_references {
          project_id: "bigquery-public-data"
          dataset_id: "faa"
          table_id: "us_airports"
        }
      }
    }
  }
}



### Create Conversation

In [ ]:
# Initialize request argument(s)
data_agent_id = "data_agent_bq_0807" # @param {type:"string"}
conversation_id = "conversation_bq_0807" # @param {type:"string"}

conversation = geminidataanalytics.Conversation()
conversation.agents = [f'projects/{billing_project}/locations/global/dataAgents/{data_agent_id}']
conversation.name = f"projects/{billing_project}/locations/global/conversations/{conversation_id}"

request = geminidataanalytics.CreateConversationRequest(
    parent=f"projects/{billing_project}/locations/global",
    conversation_id=conversation_id,
    conversation=conversation,
)

# Make the request
response = data_chat_client.create_conversation(request=request)

# Handle the respe
print(response)

name: "projects/cloud-raspberry-e2e-testing/locations/global/conversations/conversation_bq_0807"
agents: "projects/cloud-raspberry-e2e-testing/locations/global/dataAgents/data_agent_bq_0807"
create_time {
  seconds: 1752057353
  nanos: 904346000
}
last_used_time {
  seconds: 1752057354
  nanos: 231466000
}



# Chat request
1. Chat request with Conversation reference
2. Chat request Data Agent
3. Chat request with Inline Context

## [Stateful] Chat request with conversation reference

In [ ]:

# Create a request containing a single user message -- your question.
question = "Make a bar graph for the top 5 states by the total number of airports" # @param {type:"string"}
messages = [geminidataanalytics.Message()]
messages[0].user_message.text = question

data_agent_id = "data_agent_bq_0807" # @param {type:"string"}
conversation_id = "conversation_bq_0807" # @param {type:"string"}

# Create a conversation_reference
conversation_reference = geminidataanalytics.ConversationReference()
conversation_reference.conversation = f"projects/{billing_project}/locations/global/conversations/{conversation_id}"
conversation_reference.data_agent_context.data_agent = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"
# conversation_reference.data_agent_context.credentials = credentials # uncomment this if you are using looker datasource

# Form the request
request = geminidataanalytics.ChatRequest(
    parent = f"projects/{billing_project}/locations/global",
    messages = messages,
    conversation_reference = conversation_reference
)

# Make the request
stream = data_chat_client.chat(request=request)

# Handle the response
for response in stream:
    show_message(response)


## [Stateless] Chat request with data_agent

In [ ]:
# Create a request containing a single user message -- your question.
question = "Make a bar graph for the top 5 states by the total number of airports" # @param {type:"string"}
messages = [geminidataanalytics.Message()]
messages[0].user_message.text = question

data_agent_id = "data_agent_bq_0107" # @param {type:"string"}

data_agent_context = geminidataanalytics.DataAgentContext()
data_agent_context.data_agent = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"
# data_agent_context.credentials = credentials # uncomment this if you are using looker datasource

# Form the request
request = geminidataanalytics.ChatRequest(
    parent=f"projects/{billing_project}/locations/global",
    messages=messages,
    data_agent_context = data_agent_context
)

# Make the request
stream = data_chat_client.chat(request=request)

# Handle the response
for response in stream:
    show_message(response)


## [Stateless] Chat request with Inline Context

In [ ]:
# Create a request containing a single user message -- your question.
question = "Make a bar graph for the top 5 states by the total number of airports" # @param {type:"string"}
messages = [geminidataanalytics.Message()]
messages[0].user_message.text = question

request = geminidataanalytics.ChatRequest(
    inline_context=inline_context,
    parent=f"projects/{billing_project}/locations/global",
    messages=messages,
)

# Make the request
stream = data_chat_client.chat(request=request)

# Handle the response
for response in stream:
    show_message(response)


# Stateless Multi-turn Conversation

In [ ]:
# @title Multi-turn Conversation

# Re-used across requests to track previous turns
conversation_messages = []

data_agent_id = "data_agent_bq_0107" # @param {type:"string"}

data_agent_context = geminidataanalytics.DataAgentContext()
data_agent_context.data_agent = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"
# data_agent_context.credentials = credentials # uncomment this if you are using looker datasource

# Helper function for calling the API
def multi_turn_Conversation(msg):

  message = geminidataanalytics.Message()
  message.user_message.text = msg

  # Send multi-turn request by including previous turns, plus new message
  conversation_messages.append(message)

  request = geminidataanalytics.ChatRequest(
      parent=f"projects/{billing_project}/locations/global",
      messages=conversation_messages,
      data_agent_context=data_agent_context,

      # uncomment the line below when using chat with inline_context
      # inline_context=inline_context
  )

  # Make the request
  stream = data_chat_client.chat(request=request)

  # Handle the response
  for response in stream:
    show_message(response)
    conversation_messages.append(response)

In [ ]:
# Send first-turn request
multi_turn_Conversation("List of the top 5 states by the total number of airports")

In [ ]:
# Send follow-up-turn request
multi_turn_Conversation("Can you please make it a pie chart?")

# Other DataAgent Service Methods

In [ ]:
# @title Get Data Agent

# Initialize request argument(s)

data_agent_id = "data_agent_bq_0107" # @param {type:"string"}
request = geminidataanalytics.GetDataAgentRequest(
    name=f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}",
)

# Make the request
response = data_agent_client.get_data_agent(request=request)

# Handle the response
print(response)


In [ ]:
# @title List Data Agent

# Initialize request argument(s)
request = geminidataanalytics.ListDataAgentsRequest(
    parent=f"projects/{billing_project}/locations/global",
)

# Make the request
page_result = data_agent_client.list_data_agents(request=request)

# Handle the response
for response in page_result:
    print(response)


In [ ]:
# @title Update Data Agent
from google.protobuf import field_mask_pb2

# Initialize request argument(s)

data_agent_id = "data_agent_bq_0107" # @param {type:"string"}

data_agent = geminidataanalytics.DataAgent()
data_agent.data_analytics_agent.published_context = published_context
data_agent.name = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"
data_agent.description = "This is the my new updated description."


update_mask = field_mask_pb2.FieldMask(paths=['description', 'data_analytics_agent.published_context'])

request = geminidataanalytics.UpdateDataAgentRequest(
    data_agent=data_agent,
    update_mask=update_mask,
)

try:
    # Make the request
    data_agent_client.update_data_agent(request=request)
    print("Data Agent Updated")
except Exception as e:
    print(f"Error updating Data Agent: {e}")


In [ ]:
# @title Set IAM Policy for Data Agent

''' PLEASE NOTE THIS API CALLS OVERRIDES EXISTING PERMISSION FOR THE RESOURCE.
For preserving existing policy in practise call Get IAM policy to fetch existing policy and pass it along with additional changes
in the call to Set IAM Policy
'''

# Inputs
data_agent_id = "shared_agent_26_06" # @param {type:"string"}
users = "abc@google.com, xyx@google.com" # @param {type:"string"}
role = "roles/geminidataanalytics.dataAgentEditor" # @param {type:"string"}

# Hardcoded to match working HTTP example
location = "global"

resource = f"projects/{billing_project}/locations/{location}/dataAgents/{data_agent_id}"

# Construct the IAM policy
binding = policy_pb2.Binding(
    role=role,
    members= [f"user:{i.strip()}" for i in users.split(",")]
)

policy = policy_pb2.Policy(bindings=[binding])

# Create the request
request = iam_policy_pb2.SetIamPolicyRequest(
    resource=resource,
    policy=policy
)

# Send the request
try:
    response = data_agent_client.set_iam_policy(request=request)
    print("IAM Policy set successfully!")
    print(f"Response: {response}")
except Exception as e:
    print(f"Error setting IAM policy: {e}")

In [ ]:
# @title GET IAM Policy for Data Agent

# Initialize request argument(s)

data_agent_id = "shared_agent_26_06" # @param {type:"string"}

resource = f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}"
request = iam_policy_pb2.GetIamPolicyRequest(
            resource=resource,
        )
try:
      response = data_agent_client.get_iam_policy(request=request)
      print("IAM Policy fetched successfully!")
      print(f"Response: {response}")
except Exception as e:
    print(f"Error setting IAM policy: {e}")

In [ ]:
# @title [Soft Delete] Delete Data Agent

# Initialize request argument(s)

data_agent_id = "da_1606" # @param {type:"string"}

request = geminidataanalytics.DeleteDataAgentRequest(
    name=f"projects/{billing_project}/locations/global/dataAgents/{data_agent_id}",
)

try:
    # Make the request
    data_agent_client.delete_data_agent(request=request)
    print("Data Agent Deleted")
except Exception as e:
    print(f"Error deleting Data Agent: {e}")


# Other DataChat Service Methods

In [ ]:
# @title Get Conversation

conversation_id = "conversation_bq_0107" # @param {type:"string"}

request = geminidataanalytics.GetConversationRequest(
     name = f"projects/{billing_project}/locations/global/conversations/{conversation_id}"
)

# Make the request
response = data_chat_client.get_conversation(request=request)

# Handle the respe
print(response)

In [ ]:
# @title List Conversation

request = geminidataanalytics.ListConversationsRequest(
    parent=f"projects/{billing_project}/locations/global",
)

# Make the request
response = data_chat_client.list_conversations(request=request)

# Handle the respe
print(response)

In [ ]:
# @title List Messages

conversation_id = "conversation_bq_2006" # @param {type:"string"}

request = geminidataanalytics.ListMessagesRequest(
    parent=f"projects/{billing_project}/locations/global/conversations/{conversation_id}",
)

# Make the request
response = data_chat_client.list_messages(request=request)

# Handle the response
print(response)